# Assignment 1: EXPLORE PANDAS FOR A LARGE DATASET


We will explore CMS (Centers for Medicare & Medicade Services) 2018 Program Year Open Payments Dataset.




<em>Open Payments, which is managed by the Centers for Medicare & Medicaid Services (CMS), is a national disclosure program created by the Affordable Care Act (ACA). The program promotes transparency and accountability by helping consumers understand the financial relationships between pharmaceutical and medical device industries, and physicians and teaching hospitals. These financial relationships may include consulting fees, research grants, travel reimbursements, and payments made from the industry to medical practitioners.
It is important to note that financial ties between the health care industry and health care providers do not necessarily indicate an improper relationship.
This document provides a guide to how CMS publishes the informational data gathered by Open Payments for public use. It explains the sources of the data, the data files that are available from CMS, and the fields contained in each data file. View the data and learn more about the Open Payments program by visiting https://www.cms.gov/openpayments/index.html."</em>
For more details on the data https://www.cms.gov/OpenPayments/Downloads/OpenPaymentsDataDictionary.pdf

Data Link Download here
https://www.cms.gov/OpenPayments/Explore-the-Data/Dataset-Downloads

Summary of the data is here
https://openpaymentsdata.cms.gov/summary

The files include information about general payments, research payments
The general payments information is in OP_DTL_GNRL_PGYR2018_P01172020.csv which is ~ 6.2GB, a large dataset in this case. 
Our working assumption is that each row corresponds to a payment and we'll ignore any row with an NaN value.
We will try to use Pandas to answer some exploratory questions:
1. What was the total payments in 2018?
2. What were the top 10 states receiving the most total amount of payments?
3. What were the top 10 states with the highest average amount of payment? **(Assignment 1)**
4. What were the 3 top physician specialty with the highest average amount of payments in CA and FL? **(Assignment 1)**

**References**
1. pandas read_csv manual
- https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html
2. dataquest tutorial on how to use pandas with big data
- https://www.dataquest.io/blog/pandas-big-data/


In [1]:
import pandas as pd

In [2]:
!ls

OP_DTL_GNRL_PGYR2018_P01172020.csv
OP_DTL_OWNRSHP_PGYR2018_P01172020.csv
OP_DTL_RSRCH_PGYR2018_P01172020.csv
OP_PGYR2018_README_P01172020.txt
OP_REMOVED_DELETED_PGYR2018_P01172020.csv
Pandas_large_dataset.ipynb
Pandas_large_dataset.ipynb.zip


In [ ]:
#-This would take forever
df = pd.read_csv("OP_DTL_GNRL_PGYR2018_P01172020.csv",low_memory=False)

In [3]:
#-Read only the first 100 rows and explore the columns
df = pd.read_csv("OP_DTL_GNRL_PGYR2018_P01172020.csv", nrows=100)
print(df.columns)

Index(['Change_Type', 'Covered_Recipient_Type', 'Teaching_Hospital_CCN',
       'Teaching_Hospital_ID', 'Teaching_Hospital_Name',
       'Physician_Profile_ID', 'Physician_First_Name', 'Physician_Middle_Name',
       'Physician_Last_Name', 'Physician_Name_Suffix',
       'Recipient_Primary_Business_Street_Address_Line1',
       'Recipient_Primary_Business_Street_Address_Line2', 'Recipient_City',
       'Recipient_State', 'Recipient_Zip_Code', 'Recipient_Country',
       'Recipient_Province', 'Recipient_Postal_Code', 'Physician_Primary_Type',
       'Physician_Specialty', 'Physician_License_State_code1',
       'Physician_License_State_code2', 'Physician_License_State_code3',
       'Physician_License_State_code4', 'Physician_License_State_code5',
       'Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name',
       'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID',
       'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name',
       'Applicable_Manufacturer_o

## 1. What was the total payments in 2018?

In [4]:
#%%timeit -n1 -r3 #<-delete the leading # to profile how long this cell block runs
df = pd.read_csv('OP_DTL_GNRL_PGYR2018_P01172020.csv',usecols=['Total_Amount_of_Payment_USDollars'])
s = df.sum()


In [5]:
print(f'The total of payments in 2018 was {s[0]}')

The total of payments in 2018 was 3009122433.1200004


**We can also use chunksize which returns a generator of dataframes**

In [6]:
#%%timeit -n1 -r3 #<-delete the leading # to profile how long this cell block runs
dfs = pd.read_csv('OP_DTL_GNRL_PGYR2018_P01172020.csv',usecols=['Total_Amount_of_Payment_USDollars'],chunksize=100000)
s = 0
for df in dfs:
    s = s + df.sum()

In [7]:
print(f'The total of payments in 2018 was {s[0]}')

The total of payments in 2018 was 3009122433.120002


## What were the top 10 states receiving the most total amount of payments?

In [8]:
cols_to_use = ['Recipient_State', 'Total_Amount_of_Payment_USDollars']

In [9]:
df = pd.read_csv('OP_DTL_GNRL_PGYR2018_P01172020.csv',usecols=cols_to_use)

In [10]:
# explore memory usage for this dataframe
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10815629 entries, 0 to 10815628
Data columns (total 2 columns):
Recipient_State                      object
Total_Amount_of_Payment_USDollars    float64
dtypes: float64(1), object(1)
memory usage: 691.1 MB


In [11]:
# by default when pandas see a columns with non numeric type or mixed type, it'll save that column as object
# In this case the recipient_state is stored as object, requiring memory usage of 691.1MB
# if we know the total different unique value for a column is less than the total number of rows then
# we can change that column type to category
# in this case we reduce memory usage to 92.8MB
df.Recipient_State = df.Recipient_State.astype('category')
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10815629 entries, 0 to 10815628
Data columns (total 2 columns):
Recipient_State                      category
Total_Amount_of_Payment_USDollars    float64
dtypes: category(1), float64(1)
memory usage: 92.8 MB


In [15]:
# Alternatively, if we can anticipate the column type beforehand
# we can use dtype parameter to specify the type of each of the imported columns
types = ['category','float64']
df = pd.read_csv('OP_DTL_GNRL_PGYR2018_P01172020.csv',usecols=cols_to_use,dtype=dict(zip(cols_to_use,types)))

In [16]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10815629 entries, 0 to 10815628
Data columns (total 2 columns):
Recipient_State                      category
Total_Amount_of_Payment_USDollars    float64
dtypes: category(1), float64(1)
memory usage: 92.8 MB


In [17]:
df.groupby('Recipient_State').sum().sort_values(by='Total_Amount_of_Payment_USDollars',ascending=False)[0:10]

,Total_Amount_of_Payment_USDollars
Recipient_State,
CA,8.270775e+08
TX,2.131902e+08
NY,2.013341e+08
MA,1.590107e+08
FL,1.583672e+08
PA,1.325657e+08
OH,1.028195e+08
IL,8.819981e+07
MI,8.643460e+07


## **What were the top 10 states with the highest average amount of payment? (Assignment 1)**
Submit your answers on canvas

In [17]:
def readData(filename, columns, columns_type):
    """Read in only columns and specify columns type from filename"""
    return pd.read_csv(filename,usecols=columns,dtype=dict(zip(columns,columns_type)))

filename = 'OP_DTL_GNRL_PGYR2018_P01172020.csv'
cols_to_use = ['Recipient_State', 
               'Total_Amount_of_Payment_USDollars',
               'Number_of_Payments_Included_in_Total_Amount',
               'Physician_Specialty']

types = ['category','float64','float64','category']

df = readData(filename,cols_to_use,types)

In [20]:
print(df.columns) # find the order of column

Index(['Recipient_State', 'Physician_Specialty',
       'Total_Amount_of_Payment_USDollars',
       'Number_of_Payments_Included_in_Total_Amount'],
      dtype='object')


In [21]:
df.columns = ['state','specialty','total_amount','payment_count']

In [27]:
df_state = df.groupby('state').agg({'total_amount':'sum','payment_count':'sum'})
df_state['average_amount'] = df_state['total_amount'].div(df_state['payment_count'])

In [29]:
df_state.sort_values(by='average_amount',ascending=False)[0:10]

,total_amount,payment_count,average_amount
state,,,
ND,1.877303e+07,14906.0,1259.428015
MA,1.590107e+08,163137.0,974.706810
VT,1.698693e+06,1893.0,897.355219
MN,4.051842e+07,47694.0,849.549619
CA,8.270775e+08,1131008.0,731.274695
PW,4.751420e+03,9.0,527.935556
DC,1.859181e+07,39858.0,466.451212
WA,5.361370e+07,126284.0,424.548610
CO,5.631983e+07,134555.0,418.563613


PW isn't one of 50 states and the payment count is only 9. This might be an outlier and we'd need further analysis if we were doing for a thorough report.

## **What were the 3 top physician specialties with the highest average amount of payments in CA and FL? **(Assignment 1)**
Submit your answers on canvas

A general practice is to package codes that would be repeated more than twice into a function

In [41]:
def findTopStateSpecialty(df,state,top=3):
    df_state = df.loc[df['state']==state].copy() # copy to avoid pass in by values
    df_state = df_state.groupby('specialty').agg({'total_amount':'sum','payment_count':'sum'})
    df_state['average_amount'] = df_state['total_amount'].div(df_state['payment_count'])
    return df_state.sort_values(by='average_amount',ascending=False)[0:top]


In [42]:
findTopStateSpecialty(df,'CA',3)

,total_amount,payment_count,average_amount
specialty,,,
Allopathic & Osteopathic Physicians|Plastic Surgery|Plastic Surgery Within the Head and Neck,16714669.12,127.0,131611.567874
Allopathic & Osteopathic Physicians|Clinical Pharmacology,255583.90,28.0,9127.996429
Dental Providers|Dentist|Endodontics,12337210.23,1775.0,6950.540975


In [43]:
findTopStateSpecialty(df,'FL',3)

,total_amount,payment_count,average_amount
specialty,,,
Allopathic & Osteopathic Physicians|Surgery|Surgery of the Hand,1603286.88,192.0,8350.452500
Allopathic & Osteopathic Physicians|Pathology|Blood Banking & Transfusion Medicine,224257.49,58.0,3866.508448
Allopathic & Osteopathic Physicians|Orthopaedic Surgery|Orthopaedic Trauma,1115548.73,552.0,2020.921612
